In [3]:
import numpy as np
import simtk.openmm as mm
import simtk.openmm.app as app
from simtk.openmm import unit
from simtk.openmm.openmm import NonbondedForce
import os
from openmmtools.testsystems import AlanineDipeptideExplicit
import sys

In [4]:
def percentageString(progress, total, divisions):
    length=total/divisions
    stage=int(progress//length)
    token=">"
    if divisions==stage:
        token="X"
    if divisions-stage<0:
        return "Overcompletion!"
    return "["+"="*stage+token+" "*(divisions-stage)+"]"

In [5]:
name="alad-015M-15A"
run="NOCHARMM-NPT"; fetch_run="NOCHARMM-NVT"
resultsdir="results/"+"{}-run-{}".format(name,run)
fetchdir="results/"+"{}-run-{}".format(name,fetch_run)
if not os.path.exists(resultsdir):
    os.mkdir(resultsdir)
    print("done")

done


In [6]:
pdb = app.PDBFile(fetchdir+"/equilibrated-NVT_{}.pdb".format(name))

In [7]:
constraints = app.HBonds
nonbonded_cutoff = 1.0*unit.nanometer
switch_distance = 0.8*unit.nanometer
nonbonded_method = app.PME
implicit_solvent = False
solvated = True
hydrogen_mass = None

In [8]:
friction = 1.0 / unit.picoseconds
pressure = 1.0 * unit.atmosphere
temperature = 298*unit.kelvin
timestep = 2.0 * unit.femtosecond

In [9]:
create_system_kwargs = dict(
    removeCMMotion=True,
    nonbondedMethod=nonbonded_method,
    nonbondedCutoff=nonbonded_cutoff,
    switchDistance=switch_distance,
    constraints=constraints,
    hydrogenMass=hydrogen_mass,
    rigidWater=True,
)

In [10]:
alanine=AlanineDipeptideExplicit(**create_system_kwargs)

In [11]:
system = alanine.system
# Add barostat
barostat = mm.MonteCarloBarostat(pressure, temperature)
system.addForce(barostat)

positions = pdb.getPositions()
topology = alanine.topology

integrator = mm.LangevinIntegrator(temperature , friction , timestep)
integrator.setConstraintTolerance(1e-7)

platform = mm.Platform.getPlatformByName("OpenCL")
platform_properties = {"DeviceIndex": "0", "Precision": "mixed"}

In [12]:
simulation = mm.app.Simulation(topology, system, integrator, platform, platform_properties)
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(temperature)

In [13]:
simulation.minimizeEnergy()
simulation.reporters.append(app.DCDReporter(resultsdir+'/NPT-equilibration_{}.dcd'.format(name), 1000))
simulation.reporters.append(app.StateDataReporter(resultsdir+'/scalars_{}.csv'.format(name), 
    1000, time=True, potentialEnergy=True, totalEnergy=True,
    temperature=True, speed=True, elapsedTime=True, volume=True))

In [18]:
step_size = 100
n_steps = 500
#20000 ps
loops = 1
box = simulation.context.getState().getPeriodicBoxVectors()
print("{:.4f}".format(box[0].x))

3.2853


In [19]:
print(f"NPT equilibration will last: {timestep.in_units_of(unit.picosecond)*step_size*n_steps*loops}")

NPT equilibration will last: 100.0 ps


In [20]:
for loop in range(loops):
    for ix in range(n_steps):
        simulation.step(step_size)
        box = simulation.context.getState().getPeriodicBoxVectors()
        percentageBar=percentageString(ix, n_steps-1, 50)
        print("\rBox: {:.8f} , {:.5f} % {}".format(box[0].x, ix/n_steps*100, percentageBar), end="")

Box: 2.87917308 , 99.80000 % [=================================================> ]

In [21]:
crd = simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions()
app.pdbfile.PDBFile.writeFile(topology, crd, open(resultsdir+'/equilibrated-NPT_{}.pdb'.format(name), 'w'))
box = np.array(simulation.context.getState().getPeriodicBoxVectors().value_in_unit(unit.nanometer))
a,b,c = box[0,0],box[1,1],box[2,2]
np.savetxt(resultsdir+'/NPT-box-lengths_{}.txt'.format(name),np.array((a,b,c)))